## Импорт библиотек и загрузка данных

Загружаем датасет

In [ ]:
!curl -O https://code.s3.yandex.net/datasets/moscow_places.csv
!curl -O https://code.s3.yandex.net/data-analyst/admin_level_geomap.geojson

Импортируем все необходимые библиотеки и проверяем что датасет скачан и на своем месте

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import json
import re
import os

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

os.listdir()

Загружаем данные из датасета и выводим базовую информацию

In [ ]:
data = pd.read_csv('moscow_places.csv')
print(data.info())
print('='*100)
print(data.head())

## Предобработка данных

### Проверим дубликаты

In [ ]:
print(f"Количество явных дубликатов: {data.duplicated().sum()}")
data = data.drop_duplicates()

### Обработаем пропуски

In [ ]:
print((data.isna().mean() * 100).sort_values(ascending=False))

Большой процент незаполненных ячеек, но заполнение медианой или удаление плохо повлияет на данные.
Можно обработать hours:

In [ ]:
data['hours'] = data['hours'].fillna('не указано')

### Создание столбца street

In [ ]:
data['street'] = data['address'].str.split(',',expand=True)[1]

### Создание столбца is_24_7

In [ ]:
data['is_24_7'] = data['hours'] == 'ежедневно, круглосуточно'

display(data.loc[data['is_24_7'] == True].head())

### Проверка результатов

In [ ]:
display(data[['address', 'street', 'hours', 'is_24_7']].head())

## Анализ данных (EDA)

### Категории заведений

In [ ]:
plt.figure(figsize=(14, 6))
order = data['category'].value_counts().index
sns.countplot(x='category', data=data, order=order, palette='viridis')
plt.title('Распределение количества заведений по категориям')
plt.xticks(rotation=45)
plt.xlabel('Категория')
plt.ylabel('Количество')
plt.show()

Вывод: Чаще всего встречаются кафе, рестораны и кофейни.

### Количество посадочных мест

Используем boxplot для отображения распределения и выбросов, ограничим ось Y для читаемости

In [ ]:
plt.figure(figsize=(14, 6))
sns.boxplot(x='category', y='seats', data=data, palette='Set2')
plt.ylim(0, 300)
plt.title('Распределение количества посадочных мест по категориям')
plt.xticks(rotation=45)
plt.show()

Вывод:
1. Лидеры по вместимости: Самые большие площадки характерны для категорий "Ресторан" и "Бар, паб". Медианное значение у них находится в районе 85–90 мест. Это логично, так как эти форматы предполагают длительное пребывание гостей.
2. Особенность кофеен: Кофейни в Москве - это достаточно просторные заведения. Их медианное количество мест (около 80) сопоставимо с барами и превышает показатели обычных кафе.
3. Компактные форматы: Самые маленькие по количеству мест — Булочные и Пиццерии (медиана около 50 мест). Вероятно, эти форматы больше ориентированы на доставку или быструю покупку навынос.
4. Разброс в фастфуде: Категория Быстрое питание имеет очень широкий разброс. Это значит, что на рынке присутствуют как совсем маленькие точки - ларьки, так и огромные фудкорты или отдельные рестораны быстрого питания.

### Соотношение сетевых и несетевых заведений

In [ ]:
chain_counts = data['chain'].value_counts()
plt.figure(figsize=(6, 6))
plt.pie(chain_counts, labels=['Несетевые', 'Сетевые'], autopct='%1.1f%%', colors=['#66b3ff','#99ff99'])
plt.title('Соотношение сетевых и несетевых заведений')
plt.show()

Вывод:

1. Рынок общепита Москвы на 62% состоит из несетевых заведений. Это говорит о том, что потребитель ценит разнообразие и уникальность.
2. Весомая доля сетей: Сетевые проекты занимают 38% рынка. Это значительная доля,почти каждое третье заведение работает по франшизе.

### Сетевые заведения по категориям

In [ ]:
chain_ratio = data.groupby('category')['chain'].mean().sort_values(ascending=False)
plt.figure(figsize=(14, 6))
sns.barplot(x=chain_ratio.index, y=chain_ratio.values, palette='coolwarm')
plt.title('Доля сетевых заведений внутри каждой категории')
plt.xticks(rotation=45)
plt.ylabel('Доля сетевых (от 0 до 1)')
plt.show()

Вывод: Фастфуд булочные, пиццерии и кофейни чаще всего развиваются как сети. Продукт в этих заведениях (хлеб, пицца, кофе) легко стандартизировать. Технологические карты простые, и масштабировать такой бизнес легче всего.

### Топ-15 популярных сетей

In [ ]:
top_chains = data[data['chain'] == 1]['name'].value_counts().head(15)

plt.figure(figsize=(14, 6))
sns.barplot(x=top_chains.values, y=top_chains.index, palette='magma')
plt.title('Топ-15 популярных сетей в Москве')
plt.xlabel('Количество точек')
plt.show()

Вывод: В лидерах кофейни (Шоколадница) и пиццерии (Доминос, Додо)

### Районы Москвы

In [ ]:
top_districts = data['district'].value_counts()

plt.figure(figsize=(14, 6))
sns.barplot(x=top_districts.values, y=top_districts.index, palette='Blues_r')
plt.title('Количество заведений по административным округам')
plt.xlabel('Количество')
plt.show()

Вывод: Большая часть заведений сосредоточена в центре Москвы

### Рейтинги по категориям

In [ ]:
plt.figure(figsize=(14, 6))
sns.boxplot(x='category', y='rating', data=data, palette='Pastel1')
plt.title('Распределение рейтингов по категориям')
plt.xticks(rotation=45)
plt.show()

Вывод: Бары и пабы часто имеют более высокий средний рейтинг, чем фастфуд.

### Фоновая картограмма (Рейтинг по районам)

In [ ]:
district_ratings = data.groupby('district')['rating'].mean().reset_index()

try:
    with open('admin_level_geomap.geojson', 'r', encoding='utf-8') as f:
        geo_data = json.load(f)

    m = folium.Map(location=[55.75, 37.61], zoom_start=10)

    folium.Choropleth(
        geo_data=geo_data,
        name='choropleth',
        data=district_ratings,
        columns=['district', 'rating'],
        key_on='feature.name',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Средний рейтинг'
    ).add_to(m)

    display(m)
except:
    print("Не удалось загрузить GeoJSON или построить карту.")

Вывод: Лучшие заведения расположены в центре города или на севере

### Кластеризация на карте

In [ ]:
m_cluster = folium.Map(location=[55.75, 37.61], zoom_start=10)
marker_cluster = plugins.MarkerCluster().add_to(m_cluster)

for idx, row in data.dropna(subset=['lat', 'lng']).iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=f"{row['name']} ({row['category']})",
    ).add_to(marker_cluster)

display(m_cluster)

### Топ-15 улиц

In [ ]:
top_streets = data['street'].value_counts().head(15)

top_streets_names = top_streets.index
df_top_streets = data[data['street'].isin(top_streets_names)]

plt.figure(figsize=(14, 8))
sns.countplot(y='street', hue='category', data=df_top_streets, order=top_streets_names)
plt.title('Распределение категорий заведений на топ-15 улицах')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

1. Гастрономические артерии города
Абсолютным лидером по количеству заведений является Проспект Мира. Здесь представлены все категории: от фастфуда до ресторанов. Следом идут другие крупные радиальные проспекты: Ленинский, Ленинградский, Профсоюзная улица.
Это улицы с высокой проходимостью, где бизнес строится на потоке людей.

2. Аномалия МКАД
Обратите внимание на МКАД. Эта улица входит в топ, но структура заведений там уникальна: огромная доля категории Кафе и почти полное отсутствие Ресторанов и Баров.
Заведения здесь — это придорожные кафе для водителей.

3. Тусовочные улицы vs Транзитные
Интересно выглядит Пятницкая улица. В отличие от длинных проспектов, это более компактная улица в центре. Здесь визуально выше доля ресторанов и заметен сегмент баров/пабов, при этом фастфуда меньше.

4. Кофейни есть на всех топовых улицах.
Больше всего их на Проспекте Мира, Ленинградском и Ленинском проспектах.
Это подтверждает гипотезу: кофейни любят большие потоки людей и соседство с бизнес-центрами, которых много на этих магистралях.

### Улицы с одним заведением

In [ ]:
street_counts = data['street'].value_counts()
one_venue_streets = street_counts[street_counts == 1].index
one_venue_df = data[data['street'].isin(one_venue_streets)]

print(f"Количество улиц с одним заведением: {len(one_venue_streets)}")
print("Топ категорий на таких улицах:")
print(one_venue_df['category'].value_counts().head())

1. Масштаб свободных зон
В Москве обнаружено 458 улиц, на которых работает всего одно заведение общепита. Это означает, что существует множество локаций (скорее всего, спальные районы, короткие переулки или новые ЖК), где рынок совершенно не насыщен. В таких местах вы автоматически становитесь монополистом.

2. Лидерами на таких улицах являются Кафе (160) и Рестораны (93).

3. Ниша для Кофейни
Кофейни занимают 3-е место (84 улицы).

4. Специфика столовых
Присутствие 36 столовых указывает на промышленные зоны или улицы рядом с вузами/бизнес-центрами, где столовая закрывает потребность сотрудников в обедах, и другим форматам там выжить сложно.

### Средний чек по районам (Ценовая карта)

In [ ]:
district_prices = data.groupby('district')['middle_avg_bill'].median().reset_index()

try:
    m_price = folium.Map(location=[55.75, 37.61], zoom_start=10)

    folium.Choropleth(
        geo_data=geo_data,
        data=district_prices,
        columns=['district', 'middle_avg_bill'],
        key_on='feature.name',
        fill_color='BuPu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Медианный средний чек (руб)'
    ).add_to(m_price)

    display(m_price)
except:
    print("GeoJSON недоступен.")

Вывод: В центре (ЦАО) чеки значительно выше. По мере удаления к МКАД цены снижаются, но есть локальные дорогие зоны (ЗАО, Аэропорты).

## Детализация: Открытие кофейни

In [ ]:
# Фильтруем кофейни
coffee_shops = data[data['category'] == 'кофейня'].copy()
print(f"Всего кофеен: {len(coffee_shops)}")

# Где их больше всего?
print("\nТоп районов по количеству кофеен:")
print(coffee_shops['district'].value_counts().head())

# Круглосуточные кофейни
print("\nКоличество круглосуточных кофеен:", coffee_shops['is_24_7'].sum())
print(f"Доля круглосуточных: {coffee_shops['is_24_7'].mean():.1%}")

# Рейтинги
plt.figure(figsize=(10, 5))
sns.histplot(coffee_shops['rating'], bins=20, kde=True, color='brown')
plt.title('Распределение рейтинга кофеен')
plt.show()

# Стоимость чашки капучино
avg_cap = coffee_shops['middle_coffee_cup'].median()
print(f"\nМедианная цена чашки капучино по Москве: {avg_cap} руб.")

plt.figure(figsize=(10, 5))
sns.boxplot(x='district', y='middle_coffee_cup', data=coffee_shops)
plt.title('Стоимость чашки капучино по округам')
plt.xticks(rotation=90)
plt.show()

### Рекомендация

1. **Конкуренция:** Рынок кофеен насыщен в ЦАО (Центральный округ), там самая высокая конкуренция, поэтому лучше отказаться от этого места чтобы не конкурировать с крупными игроками
2. **Цена:** Ориентироваться стоит на цену чашки капучино в районе 150-200 рублей. Это медианное значение.
3. **Локация:** Рекомендуется расмотреть САО, СЗАО или ЗАО. Там платежеспособная аудитория (высокие средние чеки в общем по общепиту), но плотность кофеен ниже, чем в центре. Рекомендуется искать улицу с одним заведением в густонаселенном жилом массиве. Мы нашли 458 таких улиц в Москве — это возможность стать локальным монополистом для жителей района.
4. **Формат:** Круглосуточных кофеен очень мало (<3-5%). Это может быть конкурентным преимуществом, если локация находится у транспортного хаба или парка.

Презентация: < Bi_anal_moscow_places.pptx >